In [8]:
# data wrangling
import pandas as pd
import numpy as np

# plotting
import matplotlib.pyplot as plt

import time

from functools import reduce

# sklearn
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier

# local files
import prediction_metrics as pm

***Load Data***

In [4]:
DATA_FILEPATH = "../data/all_hourly_data.h5"

patients = pd.read_hdf(DATA_FILEPATH, "patients")
# vitals_labs_mean = pd.read_hdf(DATA_FILEPATH, "vitals_labs_mean")
interventions = pd.read_hdf(DATA_FILEPATH, "interventions")

***prepare data***

In [3]:
"""
Task Formulation: predict whether a patient will die, given the first 24 hours of their stay
"""

# SETTINGS
window_size = 24  # the first WINDOW_SIZE hours of the patient's stay
gap_time = 6  # the number of hours the patient lived at least after the first WINDOW_SIZE hours (to avoid label leakage, see MIMIC-III Extract paper)
test_size = 0.2  # proportion of the data that wil lbe used for testing
val_size = 0.125  # proportion of the training data that will be used for validation
random_state = 42  # random state is used to set a seed for randomness, which is only relevant for reproducibility purposes
max_missing = 0.8  # maximum percentage of missing values after forward fill for a measurement to be dropped

In [38]:
interventions_new = interventions.reset_index(level=["hadm_id", "icustay_id"], drop=True)
interventions_new = interventions_new.groupby(level="subject_id").max()
interventions_new.head()


,vent,vaso,adenosine,dobutamine,dopamine,epinephrine,isuprel,milrinone,norepinephrine,phenylephrine,vasopressin,colloid_bolus,crystalloid_bolus,nivdurations
subject_id,,,,,,,,,,,,,,
3,1,1,0,0,1,0,0,0,1,1,0,0,1,1
4,0,0,0,0,0,0,0,0,0,0,0,0,1,1
6,0,0,0,0,0,0,0,0,0,0,0,0,0,1
9,1,1,0,0,0,0,0,0,1,1,0,0,0,0
11,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [39]:
start_time = time.time()

patients_new = patients.reset_index(level=["hadm_id", "icustay_id"], drop=True)

"""PREPARE VITALS LABES AND INTERVENTIONS"""
# get target variable (died in ICU) of patients that stayed at least GAP_TIME + WINDOW_SIZE hours in the ICU
y = patients_new["mort_icu"]

"""ADD DEMOGRAPHICS"""
X = pd.get_dummies(patients_new[["gender"]], drop_first=True)
X["age"] = patients_new["age"]
X = X.join(pd.get_dummies(patients_new["ethnicity"], drop_first=True))
X = X.join(interventions_new)

"""SPLIT DATA"""
# define train/test split based on index
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=test_size, random_state=random_state
)
X_train, X_val, y_train, y_val = train_test_split(
    X_train, y_train, test_size=val_size, random_state=random_state
)

"""RESET INDEX"""
y_train = y_train.reindex(X_train.index)
y_test = y_test.reindex(X_test.index)
y_val = y_val.reindex(X_val.index)

"""Scale data to 0 mean and 1 standard deviation"""
for column in X_train.columns:
    X_test[column] = (X_test[column] - X_train[column].mean())/X_train[column].std()
    X_val[column] = (X_val[column] - X_train[column].mean())/X_train[column].std()
    X_train[column] = (X_train[column] - X_train[column].mean())/X_train[column].std()


"""PRINT STATS"""
print("Time: %.2fs" % (time.time() - start_time))
print("Original set: %s rows, %s columns" % patients.shape)
print("Train set: %s rows, %s columns" % X_train.shape)
print("Validation set: %s rows, %s columns" % X_val.shape)
print("Test set: %s rows, %s columns" % X_test.shape)

Time: 0.21s
Original set: 34472 rows, 28 columns
Train set: 24129 rows, 56 columns
Validation set: 3448 rows, 56 columns
Test set: 6895 rows, 56 columns


***train linear regression model***

In [40]:
decision_tree_classifier_model = DecisionTreeClassifier()
decision_tree_classifier_model.fit(X_train, y_train)

DecisionTreeClassifier()

***test linear regression model***

In [41]:
y_true = y_test.to_list()
y_pred = np.round(decision_tree_classifier_model.predict(X_test))

print("TP:", pm.TP(y_true, y_pred))
print("FP:", pm.FP(y_true, y_pred))
print("TN:", pm.TN(y_true, y_pred))
print("FN:", pm.FN(y_true, y_pred))
print("Recall:", pm.recall(y_true, y_pred))

TP: 205
FP: 283
TN: 6174
FN: 233
Recall: 0.4680365296803653
